In [4]:
import os
from dotenv import load_dotenv

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import hashlib
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import langchain
load_dotenv()
embeddings = OpenAIEmbeddings()

#langchain.debug = True

In [5]:
#persist_directory = './db/chroma/api_doc'
#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


In [5]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI, ChatVertexAI
from langchain.chat_models.anthropic import ChatAnthropic
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferMemory
import LANGUAGE_PROMPT

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)


In [6]:

langchain.debug = True
persist_directory = './db/chroma/api_doc'

db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

#retriever = db.as_retriever(search_type="mmr")
#retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .3})
retriever = db.as_retriever()
examples = ['aiRecognize有哪些参数?', 'aiRecognize需要哪些权限', '请提供aiRecognize的示例代码?']
examples = ['aiRecognize有哪些参数']
examples = ['我想实现视觉相关功能有哪些API可以支持']

examples = ['aiRecognizeAPI使用相关的示例代码']
examples = ["aiRecognize使用示例代码"]

openai_api_key = os.environ.get("OPENAI_API_KEY")
ConversationalRetrievalChain.prompts = LANGUAGE_PROMPT
qa = None
# this overwrites the built-in prompt of the ConversationalRetrievalChain
ConversationalRetrievalChain.prompts = LANGUAGE_PROMPT
qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(
        model_name="gpt-3.5-turbo", openai_api_key=openai_api_key, 
        temperature="0.0", max_tokens=1024), 
        db.as_retriever(), memory=memory, verbose=True, max_tokens_limit=1024)
for doc in examples:    
    query = doc
    model_response = qa({"question": doc})    
    print(model_response['answer'])
    #time.sleep(1)

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "aiRecognize使用示例代码",
  "chat_history": []
}
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:StuffDocumentsChain] Entering Chain run with input:
{
  "input_documents": [],
  "question": "aiRecognize使用示例代码",
  "chat_history": ""
}
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "aiRecognize使用示例代码",
  "context": ""
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:StuffDocumentsChain > 3:chain:LLMChain > 4:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n\nHuman: aiRecognize使用示例代码"
  ]
}
[llm/end] [1:chain:ConversationalRetrievalChain > 2:chain:StuffDocumentsChain 

In [30]:
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA

qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=db)
query = "aiRecognize有哪些参数"
qa.run(query)

[chain/start] [1:chain:VectorDBQA] Entering Chain run with input:
{
  "query": "aiRecognize有哪些参数"
}
[chain/start] [1:chain:VectorDBQA > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:VectorDBQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "aiRecognize有哪些参数",
  "context": "kl.aiRecognize\n\nAPI名称\n\naiRecognize\n\n功能描述\n\n快驴容器增加视觉识别能力\n\n参数说明\n\n输入参数\n\nschema: 'aiRecognize',\n})\n// 调用异步 api\nif (isSupport) {\n  myapi.callAsync({\n    appId: 'value',\n    businessId: 'value',\n    isOnline: 'value',\n    cardType: 'value',\n    inputCardMode: 'value',\n\nretryCount: 'value',\n    maxScanTime: 'value',\n    flashMode: 'value',\n    capacity: 'value',\n    target: 'value',\n    strTag: 'value',\n    maxHeight: 'value',\n    maxWidth: 'value',\n    imgPath: 'value',\n\n// 创建 api\nconst myapi = wx.registerExtendedAPI({\n  supportAsync: True,\n  supportSync: False,\n  name: 'aiRecognize',\n  sco

' inputCardMode, retryCount, maxScanTime, flashMode, capacity, target, strTag, maxHeight, maxWidth, imgPath.'

In [41]:
def getFileContext(filepath):
    # 用'with'关键字自动关闭文件
    with open('filename.txt', 'r') as f:
        content = f.read()
    #print(content)
    return content

def convert_bytes(bytes, precision=2):
    """Converts bytes into a human-friendly format."""
    abbreviations = ['B', 'KB', 'MB']
    if bytes <= 0:
        return '0 B'
    size = bytes
    index = 0
    while size >= 1024 and index < len(abbreviations) - 1:
        size /= 1024
        index += 1
    return f'{size:.{precision}f} {abbreviations[index]}'

def get_file_size(filepath):
    file_size = os.path.getsize(filepath)
    return file_size

def compute_sha1_from_file(file_path):
    with open(file_path, "rb") as file:
        bytes = file.read()
        readable_hash = compute_sha1_from_content(bytes)
    return readable_hash


def compute_sha1_from_content(content):
    readable_hash = hashlib.sha1(content).hexdigest()
    return readable_hash    

In [37]:
def loadFile(filepath):
    loader = UnstructuredMarkdownLoader(filepath)
    documents = loader.load()
    file_sha1 = compute_sha1_from_file(filepath)
    file_size = get_file_size(filepath)
    return documents, file_sha1, file_size

In [34]:
file_path = './api_docs/aiRecognize.md'
chunk_size = 500
chunk_overlap = 0

In [35]:
from langchain.evaluation.qa import QAGenerateChain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


In [42]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap)
file_documents, file_sha1, file_size = loadFile(file_path)
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [43]:
#print(len(file_documents))
langchain.debug = True
persist_directory = './db/chroma_800/api_doc'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
#documents = text_splitter.split_documents(file_documents)
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in file_documents]
)
print(new_examples)

[chain/start] [1:chain:QAGenerateChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:QAGenerateChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\npage_content=\"kl.aiRecognize\\n\\nAPI名称\\n\\naiRecognize\\n\\n功能描述\\n\\n快驴容器增加视觉识别能力\\n\\n参数说明\\n\\n输入参数\\n\\n| 属性 | 类型 | 默认值 | 必填 | 说明 | 最大值 | 最小值 | 枚举值 |\\n| --- | --- | --- | --- | --- | --- | --- | --- |\\n| appId | int |  -  | True | appId | - | - | - |\\n| businessId | string |  -  | True | businessId | - | - | - |\\n| isOnline | boolean |  -  | False | isOnline | - | - | - |\\n| cardType | int |  -  | True | card

ValueError: Could not parse output: QUESTION: What is the purpose of the aiRecognize API?

ANSWER: The aiRecognize API is intended to add visual recognition capability to the Quick Donkey container.

In [34]:
llm = ChatOpenAI(temperature = 0.0)
retriever = vectordb.as_retriever(search_type="mmr")
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)
qa.run(new_examples[0]["query"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the purpose of the aiRecognize API?"
}


Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1


[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the purpose of the aiRecognize API?",
  "context": "kl.aiRecognize\n\nAPI名称\n\naiRecognize\n\n功能描述\n\n快驴容器增加视觉识别能力\n\n参数说明\n\n输入参数"
}
[llm/start] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain > 4:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\nkl.aiRecognize\n\nAPI名称\n\naiRecognize\n\n功能描述\n\n快驴容器增加视觉识别能力\n\n参数说明\n\n输入参数\nHuman: What is the purpose of the aiRecognize API?"
  ]
}
[llm/end] [1:chain:RetrievalQA > 2:chain:StuffDocumentsChain > 3:chain:LLMChain > 4:llm:ChatOpenAI] [1.10s] Exiting LLM run with output:
{
  

'The aiRecognize API is used to add visual recognition capabilities to the Fast Donkey container.'

In [35]:
langchain.debug = False

In [1]:
predictions = qa.apply(new_examples)
from langchain.evaluation.qa import QAEvalChain


NameError: name 'qa' is not defined

In [38]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(new_examples, predictions)

In [39]:
for i, eg in enumerate(new_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: What is the purpose of the aiRecognize API?
Real Answer: The aiRecognize API adds visual recognition capabilities to the Quick Donkey container.
Predicted Answer: The aiRecognize API is used to add visual recognition capabilities to the Fast Donkey container.
Predicted Grade: CORRECT

